In [1]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torchsampler import ImbalancedDatasetSampler
from yaml import load as load_yaml, FullLoader

from dataset import KIDataset, train_test_split_stratified
from models.rocket import ROCKET
from models.transformer import TransformerClassifier, LitTimeSeriesClassifier
from processor.processor import Leif
from utils.const import SEED
from utils.data import binarize
from utils.misc import set_random_state
from utils.path import config_path, log_path

set_random_state(SEED)
!conda activate eyetrackpdc

# Data parameters
BINARY_CLF = True

# Rocket parameters
NUM_KERNELS = 1000
NORMALIZE = True

# Dataloader Parameters
BATCH_SIZE = 16

### Load Configs

In [2]:
with open(config_path.joinpath('leif.yaml'), 'r') as reader:
    processor_config = load_yaml(reader, Loader=FullLoader)
with open(config_path.joinpath('transformer.yaml')) as reader:
    transformer_config = load_yaml(reader, Loader=FullLoader)

### Prepare Data

In [3]:
# Configure processor
processor = Leif(processor_config)

# Initialize Datasets
train_val_ds = KIDataset(data_processor=processor, train=True, bundle_as_trials=True)
train_ds, val_ds = train_test_split_stratified(train_val_ds, test_size=0.2)
test_ds = KIDataset(data_processor=processor, train=False, bundle_as_trials=True)

# Binarize dataset after split to make sure split is stratified w.r.t all three classes
if BINARY_CLF:
    for ds in [train_ds, val_ds, test_ds]:
        binarize(ds)

# Initialize Dataloaders
train_dl = DataLoader(train_ds,
                      batch_size=BATCH_SIZE,
                      sampler=ImbalancedDatasetSampler(train_ds, callback_get_label=lambda item: item.y))
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

loaded dataset from C:\Users\hejpa\Documents\GitHub\eye-track-pdc\data\ki\tmp\ki-dataset-trial-train
loaded dataset from C:\Users\hejpa\Documents\GitHub\eye-track-pdc\data\ki\tmp\ki-dataset-trial-test


### Initialize Model

In [4]:
# Initialize Rocket
rocket = ROCKET(c_in=train_ds.x.shape[2],
                seq_len=train_ds.x.shape[3],
                n_kernels=NUM_KERNELS,
                normalize=NORMALIZE)

# Initialize Classifier
clf = TransformerClassifier(in_features=NUM_KERNELS * 2, **transformer_config)

# Initialize lit time series classifier
model = LitTimeSeriesClassifier(encoder=rocket, decoder=clf, feature_dim=NUM_KERNELS * 2, lr=1e-5, wd=1e-1)

### Train Model

In [5]:
trainer = Trainer(accelerator='auto',
                  max_epochs=100,
                  auto_lr_find=False,
                  logger=TensorBoardLogger(save_dir=log_path),
                  log_every_n_steps=1)

trainer.fit(model, train_dl, val_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                  | Params
--------------------------------------------------
0 | encoder | ROCKET                | 54.4 K
1 | decoder | TransformerClassifier | 521 K 
2 | loss_fn | BCEWithLogitsLoss     | 0     
--------------------------------------------------
521 K     Trainable params
54.4 K    Non-trainable params
575 K     Total params
2.302     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\hejpa\Anaconda3\envs\eyetrackpdc\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
C:\Users\hejpa\Anaconda3\envs\eyetrackpdc\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
